In [1]:
from pandas import read_csv
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt


2024-06-04 13:43:49.796507: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-04 13:43:49.960051: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-04 13:43:50.582726: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-04 13:43:51.847835: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# debug
print(f'keras version: {keras.__version__}')
from tensorflow import __version__ as tensorflow_version
print(f'tensorflow version: {tensorflow_version}')

keras version: 3.3.3
tensorflow version: 2.16.1


In [3]:
# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

x_train shape: (60000, 28, 28, 1)
y_train shape: (60000,)
60000 train samples
10000 test samples


In [17]:
# Model parameters
num_classes = 10
input_shape = (28, 28, 1)

model = keras.Sequential(
    [
        keras.layers.Input(shape=input_shape),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes, activation="softmax"),
    ]
)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 24, 24, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 10, 10, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 8, 8, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 260,298 (1016.79 KB)

 Trainable params: 260,298 (1016.79 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name="acc"),
    ],
)

In [19]:
batch_size = 128
epochs = 20

callbacks = [
    keras.callbacks.ModelCheckpoint(filepath="model_at_epoch_{epoch}.keras"),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=2),
]

model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.15,
    callbacks=callbacks,
)
score = model.evaluate(x_test, y_test, verbose=0)

Epoch 1/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 20s 47ms/step - acc: 0.5282 - loss: 1.3079 - val_acc: 0.9668 - val_loss: 0.1213
Epoch 2/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 18s 46ms/step - acc: 0.9371 - loss: 0.2189 - val_acc: 0.9763 - val_loss: 0.0792
Epoch 3/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 18s 46ms/step - acc: 0.9564 - loss: 0.1461 - val_acc: 0.9847 - val_loss: 0.0540
Epoch 4/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 18s 46ms/step - acc: 0.9659 - loss: 0.1173 - val_acc: 0.9861 - val_loss: 0.0476
Epoch 5/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 18s 46ms/step - acc: 0.9702 - loss: 0.1010 - val_acc: 0.9887 - val_loss: 0.0428
Epoch 6/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 18s 45ms/step - acc: 0.9750 - loss: 0.0839 - val_acc: 0.9888 - val_loss: 0.0420
Epoch 7/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 18s 46ms/step - acc: 0.9773 - loss: 0.0763 - val_acc: 0.9893 - val_loss: 0.0413
Epoch 8/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 18s 46ms/step - acc: 0.9797 - loss: 0.0684 - val_acc: 0.9912 - val_loss: 0.0335
Epoch 9/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 

In [37]:
model.save("final_model.keras")

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step


In [4]:
model = keras.saving.load_model("final_model.keras")
predictions = model.predict(x_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step


In [5]:
p = predictions[:1]

print(type(p), p.shape)
print(type(p[0]), p[0].shape)
print(type(p[0][0]))

for i in range(10):
    print(f'{i}: {p[0][i]:.10f}')


<class 'numpy.ndarray'> (1, 10)
<class 'numpy.ndarray'> (10,)
<class 'numpy.float32'>
0: 0.0000006149
1: 0.0000000067
2: 0.0000043084
3: 0.0000005292
4: 0.0000000026
5: 0.0000000134
6: 0.0000000000
7: 0.9997456074
8: 0.0000000039
9: 0.0002489533
